# 05. 문서 유사도(Document Similarity)

## 5.1. 코사인 유사도(Cosine Similarity)

코사인 유사도는 두 벡터 간의 코사인 각도를 이용하여 구할 수 있는 두 벡터의 유사도를 의미

두 벡터의 방향이 완전히 동일한 경우에는 1의 값을 가지며,

90도의 각을 이루면 0

180도로 반대의 방향을 가지면 -1

similarity = cos(Θ)=A⋅B / ||A|| ||B|| = A, B의 내적을 구하고, A, B의 노름을 곱한 값으로 나눈다.

In [5]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [6]:
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

In [7]:
doc1=np.array([0,1,1,1])
doc2=np.array([1,0,1,1])
doc3=np.array([2,0,2,2])

In [8]:
print(cos_sim(doc1, doc2)) #문서1과 문서2의 코사인 유사도
print(cos_sim(doc1, doc3)) #문서1과 문서3의 코사인 유사도
print(cos_sim(doc2, doc3)) #문서2과 문서3의 코사인 유사도

0.6666666666666667
0.6666666666666667
1.0000000000000002


#### 유사도를 이용한 추천 시스템 구현하기

케글에서 제공되는 영화 데이터셋을 가지고 영화 추천 시스템을 만들어보겠습니다.

TF-IDF와 코사인유사도를 사용하여 줄거리 기반 영화 추천 시스템을 만들어보겠습니다.

다운로드 링크 : https://www.kaggle.com/rounakbanik/the-movies-dataset

In [1]:
import pandas as pd

In [3]:
data = pd.read_csv("./movie_dataset/movies_metadata.csv")

C:\Users\thinp\anaconda3\envs\Wyatt37\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [5]:
# 2만개만 가지고 학습할 것이다.

data = data.head(20000)

In [6]:
# TF-IDF는 null 값을 받지 못한다.

data['overview'].isnull().sum()

135

In [7]:
data['overview'] = data['overview'].fillna('')

C:\Users\thinp\anaconda3\envs\Wyatt37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
tfidf = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = tfidf.fit_transform(data['overview'])

print(tfidf_matrix.shape)

(20000, 47487)


In [10]:
# 이제 코사인 유사도를 구해줍니다.

from sklearn.metrics.pairwise import linear_kernel

In [12]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [13]:
# 여오하의 타이틀과 인덱스를 가진 테이블을 만듭니다.

indices = pd.Series(data.index, index = data['title']).drop_duplicates()
print(indices.head())

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [14]:
# 이제 코사인 유사도를 활용하여 가장 overview가 유사한 10개의 영화를 찾아내는 함수를 만듭니다.

def get_reco(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 인덱스 가져오기
    idx = indices[title]
    
    # 모든 영화에 대해 해당 영화와의 유사도 구함
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # 유사도에 따라 영화들 정렬
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)
    
    # 가장 유사한 10개의 영화 받아오기
    sim_scores = sim_scores[1:11]
    
    # 가장 유사한 10개의 영화 인덱스 받아오기
    movie_indices = [i[0] for i in sim_scores]
    
    # 가장 유사한 10개의 영화 제목 리턴
    return data['title'].iloc[movie_indices]

In [17]:
get_reco('The Dark Knight')

18252                      The Dark Knight Rises
1328                              Batman Returns
15511                 Batman: Under the Red Hood
150                               Batman Forever
19792    Batman: The Dark Knight Returns, Part 1
585                                       Batman
3095                Batman: Mask of the Phantasm
18035                           Batman: Year One
9230          Batman Beyond: Return of the Joker
10122                              Batman Begins
Name: title, dtype: object

## 5.2. 여러가지 유사도 기법

### 5.2.1. 유클리드 거리(Euclidean Distance)

유클리드 거리 잘 아시죠..? 네 그냥 두 점 사이의 직선 거리를 구하는 것입니다.

In [20]:
import numpy as np

def dist(x, y):
    return np.sqrt(np.sum((x-y)**2))

In [21]:
doc1 = np.array((2,3,0,1))
doc2 = np.array((1,2,3,1))
doc3 = np.array((2,1,2,2))
docQ = np.array((1,1,0,1))

print(dist(doc1,docQ))
print(dist(doc2,docQ))
print(dist(doc3,docQ))

2.23606797749979
3.1622776601683795
2.449489742783178


### 5.2.2. 자카드 유사도(Jaccard Similarity)

A와 B 두개의 집합이 있다고 칩시다. 교집합은 두 집합이 동시에 가지고 있는 원소의 집합을 말합니다. 즉 합집합에서 교집합의 비율을 구한다면 두 집합 A와 B의 유사도를 구할 수 있다는 것이 자카드 유사도의 원리입니다.

J(A,B) = |A∩B| / |A∪B| = |A∩B| / |A|+|B|−|A∩B| = 2|A∩B| / |A|+|B|

In [22]:
# 두 문서 모두에서 등장한 단어는 apple과 banana 2개.
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

# 토큰화를 수행합니다.
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()

# 토큰화 결과 출력
print(tokenized_doc1)
print(tokenized_doc2)

['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
['apple', 'banana', 'coupon', 'passport', 'love', 'you']


In [23]:
# 합집합을 구합니다.

union = set(tokenized_doc1).union(set(tokenized_doc2))
print(union)

{'like', 'card', 'passport', 'coupon', 'you', 'watch', 'holder', 'likey', 'everyone', 'apple', 'banana', 'love'}


In [24]:
# 교집합을 구합니다.

intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))
print(intersection)

{'banana', 'apple'}


In [25]:
print(len(intersection)/len(union))

0.16666666666666666
